In [22]:
import pandas as pd
from datetime import datetime
import time
from datetime import timedelta
import ipywidgets as widgets
import pprint
import os

%matplotlib inline

In [2]:
def convert_to_seconds(t_str):
    if isinstance(t_str, str):
        t = (datetime.strptime(t_str, '%M:%S.%f')).time()
        return ((t.hour * 60 + t.minute) * 60 + t.second + (t.microsecond / 1000000))
    return None

def convert_to_timestamp(secs):
    return str(timedelta(seconds=secs))

In [3]:
c2_log = pd.read_csv('data/LogCard.csv', header=3)

filter = (c2_log['Name'] == 'Bob') & (c2_log['Unnamed: 0'].isnull()) 
c2_log = c2_log[filter].copy()
c2_log = c2_log.dropna(axis=1, how='all').copy()
c2_log['Date'] = pd.to_datetime(c2_log['Date'])
c2_log['time_seconds'] = c2_log['Time'].apply(convert_to_seconds)
c2_log['time1_seconds'] = c2_log['Time.1'].apply(convert_to_seconds)

c2_log['workout_seq'] = c2_log.groupby(['Date', 'Time of Day', 'Workout Name']).cumcount() + 1

### Today's workout

In [15]:
today = datetime.now().date()
today

datetime.date(2018, 7, 25)

In [16]:
filter = (c2_log['Date'] == today) & (c2_log['Time.1'].isnull() == False)
c2_log[filter]['time1_seconds']

401    105.7
402    105.4
403    105.9
404    105.4
405    105.2
406    106.0
407    104.4
408     94.5
Name: time1_seconds, dtype: float64

In [17]:
filter = (c2_log['Date'] == today)
s = ' | ' + c2_log['Time.1']
c2_log[filter][['/500m']].join(s).to_clipboard(index=False)

In [18]:
# pyramid
filter = (c2_log['Date'] == today) & (c2_log['/500m'].isnull() == False)
#c2_log.loc[filter, ['Meters.1', '/500m', 'Time.1']].head(7).to_clipboard(index=False)
c2_log.loc[filter, ['Meters.1', '/500m']].to_clipboard(index=False)

In [19]:
# 3,000m, 2,500m, 2,000m
filter = (c2_log['/500m'].isnull() == False) | (c2_log['Time.1'].isnull() == False)
s = ' | ' + c2_log['Time.1']
c2_log[filter][['/500m']].join(s).head(4).to_clipboard(index=False)

### Past PP workouts

In [122]:
#workout_name = '8x500m/3:30r'         # Week 1
#workout_name = 'v250m/1:30r...7'     # Week 2
#workout_name = '4x1000m/5:00r'       # Week 3
workout_name = '5x1500m/5:00r'        # Week 1
#workout_name = '4x2000m/5:00r'       # Week 2
#workout_name = 'v3000m/5:00r...3'    # Week 3

In [123]:
filter = (c2_log['Workout Name'] == workout_name) & (c2_log['/500m'].isnull() == False) #& (c2_log['Meters.1'].isnull() == False)
#pv = c2_log[filter][['Date', '/500m', 'Time.1', 'workout_seq']].pivot(columns='Date', index='workout_seq', values='/500m')
pv = (c2_log[filter][['Meters.1', 'Date', '/500m', 'workout_seq']]
          .set_index(['Meters.1', 'workout_seq', 'Date']).unstack().sort_index(level=1)
          .reset_index().drop('workout_seq', axis=1))
pv

/Users/bob/miniconda3/envs/dev35/lib/python3.5/site-packages/pandas/core/generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Meters.1               /500m                                        
Date          2018-06-13 00:00:00 2018-07-04 00:00:00 2018-07-28 00:00:00
0         NaN             01:56.6             01:54.7             01:52.0
1        1500             01:54.5             01:55.4             01:53.4
2        1500             01:55.5             01:54.8             01:53.4
3        1500             01:57.2             01:55.4             01:53.3
4        1500             01:57.3             01:55.3             01:53.0
5        1500             01:58.3             01:52.7             01:47.0

In [124]:
def pad_cols(col_width, values, col_zero_str=''):
    s = col_zero_str
    for v in values:
        s = s + str(v).ljust(col_width, '-')
    return s + '\n'

In [125]:
col_width = 15

s = pad_cols(col_width,[str(c[1].date()) for c in pv.columns if isinstance(c[1], pd.Timestamp)], col_zero_str=('-' * col_width))

for _, row in pv.iterrows():
    row_str = pad_cols(col_width, row)
    if row_str[:3] == 'nan':
        avgs = '\nAVGs' + row_str[4:]
    else:
        s = s + row_str

s = s + avgs
os.system("echo '%s' | pbcopy" % s)

pprint.pprint(s)

('---------------2018-06-13-----2018-07-04-----2018-07-28-----\n'
 '1500-----------01:54.5--------01:55.4--------01:53.4--------\n'
 '1500-----------01:55.5--------01:54.8--------01:53.4--------\n'
 '1500-----------01:57.2--------01:55.4--------01:53.3--------\n'
 '1500-----------01:57.3--------01:55.3--------01:53.0--------\n'
 '1500-----------01:58.3--------01:52.7--------01:47.0--------\n'
 '\n'
 'AVGs-----------01:56.6--------01:54.7--------01:52.0--------\n')


### PRs

In [51]:
# 06:45.8 2000 Bob Haffner 1:41.4
filter = c2_log['Workout Name'].isin(['100m', '500m', '1000m', '2000m', '5000m'])
cols = ['Date', 'Time', 'time_seconds', '/500m']
c2_log[filter].groupby('Workout Name')[cols].apply(lambda x : x.nsmallest(n=1, columns='time_seconds')).sort_values('time_seconds')

,,Date,Time,time_seconds,/500m
Workout Name,,,,,
100m,858,2015-09-06,00:16.8,16.8,01:24.0
500m,836,2015-09-13,01:24.8,84.8,01:24.8
1000m,823,2015-09-20,03:07.0,187.0,01:33.5
2000m,686,2017-02-12,06:50.4,410.4,01:42.6
5000m,1046,2013-06-03,18:50.4,1130.4,01:53.0
